In [ ]:
# Necessary imports of functions and dictionaries
from dictionaries import *
from sample_sheet_functions import *

# User downloads sample sheet
*Requires User* 
- to pick for which tables 
- how many if 96-wells
- Project name, experiment name, additional comments

_To add_:
- User can choose csv or xlsx (xlsx is going to be coloured)


{'F901': 'AACTACAGC',
 'F902': 'TTCTGGGCA',
 'F903': 'GCACACTGT',
 'F904': 'CTAATGCAC',
 'F905': 'AGGTAACAT',
 'F906': 'GATGGTTGT',
 'F907': 'GGTCCATGC',
 'F908': 'AAGGTGTTT',
 'F909': 'CCATCAGCC',
 'F910': 'GGACTAAGA',
 'F911': 'TCATTGAAA',
 'F912': 'AGACGTGTT',
 'F913': 'CGTAGATTT',
 'F914': 'GCTCAGCTT',
 'F915': 'GGTATCCTT',
 'F916': 'AATTCGGAA',
 'F917': 'CAACACGCA',
 'F918': 'GGCCTGTCA',
 'F919': 'GAACCAATT',
 'F920': 'AGAACCATA',
 'F921': 'AGGCTATCT',
 'F922': 'GAGACCACC',
 'F923': 'GGTTAAGGC',
 'F924': 'CAGTTAATA',
 'F925': 'CACAGGCCA',
 'F926': 'TGACGGCCA',
 'F927': 'GACCTTGCT',
 'F928': 'AGATTGAGT',
 'F929': 'AGGAGTCCA',
 'F930': 'GCGATAGCA',
 'F931': 'TGGCAAGGT',
 'F932': 'CATAGCATA',
 'F933': 'CCAAGTCGT',
 'F934': 'TGCATCTGC',
 'F935': 'GACGCGTTA',
 'F936': 'AGCCTACTT',
 'F937': 'ATGGTTAGA',
 'F938': 'GCTGCGTAT',
 'F939': 'TGTAACGGC',
 'F940': 'AATAGGTGC',
 'F941': 'AACACAATC',
 'F942': 'TGCCGGTAT',
 'F943': 'TAAGTTACG',
 'F944': 'AGCTAGACC',
 'F945': 'ATTGTGTCT',
 'F946': '

# User reuploads sample sheet
#### Displays the layout of the uploaded sample sheet(s) to the user
*Requires user*
- Pick plate flavours

_In background_:
- Checks number of samples
- Checks number of blanks/controls
- Checks no samples start with underscore (_)

_To add_:
- User can upload CSV or XLSX

_Processes:_
- Read file in, extract different tables and display to users
- Associate each table to a plate flavour
- Associate dictionaries and produce generic melted sample sheet

# Prompt the user for download
*Requires user*
- Pick which machine they need

_Processes_:
- Append header to melted table